# Low Level Regularization Iteration Notebook
A notebook for gaining intuition into 4 regularization techniques (l1 and l2 weight penalties, pruning, and dropout). Implemented at a low level and using the `scikit-learn` digits dataset, this notebook relies only on `numpy`, `scikit-learn`, and `matplotlib` for plotting. 

In [ ]:
import numpy as np
# use built in dataset from scikit-learn
import sklearn.datasets as datasets
import matplotlib.pyplot as plt


def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def grad_from_sigz(sigz):
    return sigz * (1-sigz)

def stable_softmax(a):
    # stabilize as in https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
    a_ = a - np.max(a)
    softmax = np.zeros_like(a)
    
    for ck in range(a_.shape[1]):
        softmax[:,ck] = np.exp(a_[:,ck]) / (np.sum(np.exp(a_),axis=1)) 
    return softmax

def accuracy(y_pred,y):
    accuracy = np.sum(1.*np.argmax(y_pred,axis=1) == np.argmax(y,axis=1)) / np.sum(y)
    return accuracy

# random seed
my_seed = 1
np.random.seed(my_seed)


# ** Hyperparameters **
learning_rate = 3e-3
# regularization
dropout_rate = 0.02500
pruning_rate = 0.10 # prune weights with absolute value below this number of std deviations
prune_period = 2500
my_lambda_l1 = 0#1e-1#0.0 #1e-5
my_lambda_l2 = 0#1e1 #0 #0.0 #1e-5
# training duration and disp period
max_steps = 15000
disp_period = 400
init_scale = 1e-2
# ** **


# load digits dataset 64 features 10 classes 1797 samples
print("loading digits dataset") 
[x,y] = datasets.load_digits(return_X_y=True)
# normalize for unit variance
x = (x-np.mean(x)) / np.std(x)
print("input shape:",x.shape,"targets shape: ", y.shape)

# convert to one hot labels for targets
number_classes = np.max(y)+1
number_entries = y.shape[0]
onehot_y = np.zeros((number_entries,number_classes))
for counter in range(number_entries):
    onehot_y[counter,y[counter]] = 1

y = onehot_y
# split into training, validation, and test sets (60% training, 20% each to validation and test)
my_seed = 1337

num_val = round(0.2*x.shape[0])
np.random.seed(my_seed)
np.random.shuffle(x)
np.random.seed(my_seed)
np.random.shuffle(y)

x_val = x[0:num_val,:]
y_val = y[0:num_val,:]
x_test = x[num_val:2*num_val,:]
y_test = y[num_val:2*num_val,:]

x = x[2*num_val:-1,:]
y = y[2*num_val:-1,:]

# get data dimensions
dim_x = x.shape[1] # 256
dim_y = y.shape[1]
dim_h = 128


# initialize weights
wxh0 = init_scale*np.random.randn(dim_x,dim_h) 
whh1 = init_scale*np.random.randn(dim_h,dim_h)
why = init_scale*np.random.randn(dim_h,dim_y)
b0 = init_scale*np.random.randn()
b1 = init_scale*np.random.randn()
if(pruning_rate):
    mask_wxh0,mask_whh1,mask_why = np.ones_like(wxh0), np.ones_like(whh1), np.ones_like(why)

    
for ck in range(max_steps+1):
        
    # forward pass
    h0 = np.dot(x,wxh0) + b0
    h0[h0<0] = 0. # apply relu
    
    h1 = np.dot(h0,whh1) + b1
    #h1 = sigmoid(h1)
    if(dropout_rate):
        h1 *= (1.*np.random.random((h1.shape)) > dropout_rate)
        #h1 /= (1-dropout_rate)
    
    y_pred = stable_softmax(np.dot(h1,why))
    
    
    
    # cross entropy loss
    loss = - (1/len(x)) * np.sum(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    if(my_lambda_l1):
        # L2 regularization: add weight penalty
        loss += my_lambda_l1/len(x) * np.sum(np.abs(wxh0))+np.sum(np.abs(whh1))+np.sum(np.abs(why))
    if(my_lambda_l2):
        # L2 regularization: add weight penalty
        loss += my_lambda_l2/(2*len(x)) * np.sum(wxh0**2)+np.sum(whh1**2)+np.sum(why**2)
    
            
    if (ck % disp_period ==0 ): 
        # forward pass (validation data)
        h0_val = np.dot(x_val,wxh0) + b0
        h0_val[h0_val<0] = 0. # apply relu

        h1_val = np.dot(h0_val,whh1) + b1
        #h1_val = h1_val

        y_pred_val = stable_softmax(np.dot(h1_val,why))
        
        train_acc = accuracy(y_pred,y)
        val_acc = accuracy(y_pred_val,y_val)
        print("CE loss step %i = %.3e, training/validation accuracy: %.3f/%.3f"%(ck,loss,train_acc,val_acc))
        if(0):
            plt.figure(figsize=(8,8))
            plt.hist([wxh0,whh1,why],bins=16)
            plt.title("weights histogram")
            plt.show()
    
    #backward pass
    dy = (y_pred-y) / len(x)
    dwhy = np.dot(dy.T, h1).T # n by y n by h --> y by h
    dh1 = np.dot(dy, why.T) # n by y , h by y
    #dh1 = dh1 * grad_from_sigz(sigmoid(h1))
    db1 = dh1
    
    dwhh1 = np.dot(dh1.T,h0).T
    
    
    dh0 = np.dot(dh1,whh1.T)
    dh0[h0<0] = 0 #backprop through relu
    db0 = dh0
    
    dwxh0 = np.dot(dh0.T,x).T # n by h , n by x --> x by h
    
    if(my_lambda_l2):
        # add the gradients for an l2 regularization term
        for param, dparam in zip([wxh0,whh1,why],[dwxh0,dwhh1,dwhy]):
            dparam += my_lambda_l2/len(x)*(param)
            
    if(my_lambda_l1):
        # add the gradients for an l2 regularization term
        for param, dparam in zip([wxh0,whh1,why],[dwxh0,dwhh1,dwhy]):
            dparam += my_lambda_l1/len(x)*(np.sign(param))
    
    if(pruning_rate > 0.):
        # remove weights below threshold absolute value
        if(ck and ck % prune_period == 0):
            print("pruning weights...")
            for param,param_mask in zip([wxh0,whh1,why],[mask_wxh0,mask_whh1,mask_why]):
                
                std_dev = np.std(param)
                print("pruning {} of {} weights below threshold".format(\
                        np.sum(np.abs(param) < pruning_rate*std_dev), np.sum(param == param)))
                
                param[np.abs(param) < pruning_rate*std_dev] = 0
                
    
        # prevent learning on pruned connections
        dwxh0 *= mask_wxh0
        dwhh1 *= mask_whh1
        dwhy *= mask_why
    
    
    dwxh0[dwxh0>5.0],dwxh0[dwxh0<-5.0] = 5.,-5.
    dwhh1[dwhh1>5.0],dwhh1[dwhh1<-5.0] = 5.,-5.
    dwhy[dwhy>5],dwhy[dwhy<-5] = 5,-5
    
    for param, dparam in zip([wxh0,b0,whh1,b1,why],[dwxh0,db0,dwhh1,db1,dwhy]):
        param -= learning_rate * dparam
    
    
                
            
                
            
    
    